In [ ]:
"""
코드 작성에 있어 다음 레포지터리를 참고했습니다.
https://github.com/ttop32/coqui_tts_korea
참고한 위 레포지토리의 정확한 파일 주소입니다.
https://github.com/ttop32/coqui_tts_korea/blob/main/coqui_preprocessing.ipynb

학습시킬 모델의 깃허브 레포지터리 주소입니다.
https://github.com/coqui-ai/TTS

학습데이터셋 주소입니다.
https://www.kaggle.com/datasets/bryanpark/korean-single-speaker-speech-dataset?resource=download
https://github.com/goodatlas/zeroth
https://github.com/yc9701/pansori-tedxkr-corpus
"""

'\n코드 작성에 있어 다음 레포지터리를 참고했습니다.\nhttps://github.com/ttop32/coqui_tts_korea\n참고한 위 레포지토리의 정확한 파일 주소입니다.\nhttps://github.com/ttop32/coqui_tts_korea/blob/main/coqui_preprocessing.ipynb\n\n학습시킬 모델의 깃허브 레포지터리 주소입니다.\nhttps://github.com/coqui-ai/TTS\n\n학습데이터셋 주소입니다.\nhttps://www.kaggle.com/datasets/bryanpark/korean-single-speaker-speech-dataset?resource=download\nhttps://github.com/goodatlas/zeroth\nhttps://github.com/yc9701/pansori-tedxkr-corpus\n'

# 1. drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/24_Open-source-programming
!ls

/content/drive/MyDrive/24_Open-source-programming
coqui_tts_korea_load.ipynb


# 2. load dataset and preprocessing
https://www.kaggle.com/datasets/bryanpark/korean-single-speaker-speech-dataset?resource=download
https://github.com/goodatlas/zeroth
https://github.com/yc9701/pansori-tedxkr-corpus

In [ ]:
#download dataset and resample

import os
from glob import glob
import torchaudio
from pathlib import Path
from tqdm import tqdm
import csv
import shutil
import torch


target_rate=22050
download_dir="download_data/"
save_dir="resample1/"
zeroth_dir="zeroth_korean/"
tedxhr_dir="pansori_tedxkr/"
kss_dir="kss/"

data_dir_list=[zeroth_dir,tedxhr_dir,kss_dir]

for i in [download_dir,save_dir]:
    for j in data_dir_list:
        os.makedirs(i+j, exist_ok=True)

In [ ]:
#create resampled mono wav,
#dir          save_dir/wav/*.wav
def saveResampleWav(wavList,save_dir,working_dir,target_rate):
    for itemPath in tqdm(wavList):
        savePath=os.path.join(save_dir,working_dir,"wavs",Path(itemPath).stem+".wav")
        speech_array, sample_rate = torchaudio.load(itemPath)
        speech_array = torchaudio.functional.resample(speech_array, sample_rate, target_rate)
        speech_array = torch.unsqueeze(torch.mean(speech_array,axis=0),dim=0)
        os.makedirs(os.path.dirname(savePath), exist_ok=True)
        torchaudio.save(savePath, speech_array, target_rate)

In [ ]:
#create csv,
#dir           save_dir/metadata.csv
#csv format    file_name | text | text | speaker_name
def saveCsv(textList,save_dir,working_dir,csvReadLine):
    with open(os.path.join(save_dir,working_dir,"metadata.csv"), "w", encoding="utf-8") as file:
        wr = csv.writer(file, delimiter='|')
        for textPath in textList:
            with open(textPath, "r", encoding="utf-8") as f:
                lines = f.readlines()
            for line in lines:
                file_name,text,speaker_name=csvReadLine(line,textPath)
                if not os.path.exists(os.path.join(save_dir,working_dir,file_name)):
                    continue
                wr.writerow([file_name,text,text,speaker_name])

In [ ]:
#download kss # 5분 걸림
import os
os.environ['KAGGLE_USERNAME'] = ""
os.environ['KAGGLE_KEY'] = ""
#!pip install --user kaggle
#!conda install -c conda-forge kaggle -y
!kaggle datasets download -d bryanpark/korean-single-speaker-speech-dataset
!unzip korean-single-speaker-speech-dataset.zip -d $download_dir$kss_dir

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: download_data/kss/kss/4/4_0633.wav  
  inflating: download_data/kss/kss/4/4_0634.wav  
  inflating: download_data/kss/kss/4/4_0635.wav  
  inflating: download_data/kss/kss/4/4_0636.wav  
  inflating: download_data/kss/kss/4/4_0637.wav  
  inflating: download_data/kss/kss/4/4_0638.wav  
  inflating: download_data/kss/kss/4/4_0639.wav  
  inflating: download_data/kss/kss/4/4_0640.wav  
  inflating: download_data/kss/kss/4/4_0641.wav  
  inflating: download_data/kss/kss/4/4_0642.wav  
  inflating: download_data/kss/kss/4/4_0643.wav  
  inflating: download_data/kss/kss/4/4_0644.wav  
  inflating: download_data/kss/kss/4/4_0645.wav  
  inflating: download_data/kss/kss/4/4_0646.wav  
  inflating: download_data/kss/kss/4/4_0647.wav  
  inflating: download_data/kss/kss/4/4_0648.wav  
  inflating: download_data/kss/kss/4/4_0649.wav  
  inflating: download_data/kss/kss/4/4_0650.wav  
  inflating: download_data/kss/kss/4/4_0651.wav  
  inflating: d

In [ ]:
#download zeroth and tedxkr
!wget https://www.openslr.org/resources/40/zeroth_korean.tar.gz
!wget https://www.openslr.org/resources/58/pansori-tedxkr-corpus-1.0.tar.gz

!tar -xvf zeroth_korean.tar.gz -C $download_dir$zeroth_dir
!tar -xvf pansori-tedxkr-corpus-1.0.tar.gz -C $download_dir$tedxhr_dir

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
train_data_01/003/214/214_003_2609.flac
train_data_01/003/214/214_003_1678.flac
train_data_01/003/214/214_003_0313.flac
train_data_01/003/214/214_003_2181.flac
train_data_01/003/214/214_003_1025.flac
train_data_01/003/214/214_003_2810.flac
train_data_01/003/214/214_003_2194.flac
train_data_01/003/214/214_003_2219.flac
train_data_01/003/214/214_003_2898.flac
train_data_01/003/214/214_003_1806.flac
train_data_01/003/214/214_003_2875.flac
train_data_01/003/214/214_003_1232.flac
train_data_01/003/214/214_003_0087.flac
train_data_01/003/214/214_003_1487.flac
train_data_01/003/214/214_003_2338.flac
train_data_01/003/214/214_003_2645.flac
train_data_01/003/214/214_003_0947.flac
train_data_01/003/214/214_003_0855.flac
train_data_01/003/214/214_003_1099.flac
train_data_01/003/214/214_003_0639.flac
train_data_01/003/214/214_003_1134.flac
train_data_01/003/214/214_003_1522.flac
train_data_01/003/214/214_003_1975.flac
train_data_01/003/214/214_003_1949.flac
trai

In [ ]:
#preprocess tedxhr
textList=glob(download_dir+tedxhr_dir+"**/*.txt", recursive=True)
wavList=glob(download_dir+tedxhr_dir+"**/*.flac", recursive=True)
working_dir=tedxhr_dir

In [ ]:
def readLine(line,textPath):
    lineSplit = line.split(' ', 1)
    file_name=os.path.join("wavs", Path(lineSplit[0]).stem+ ".wav")
    text=lineSplit[1].strip()
    speaker_name="tedxhr_"+Path(textPath).parent.name
    return [file_name,text,speaker_name]



saveResampleWav(wavList,save_dir,working_dir,target_rate)
saveCsv(textList,save_dir,working_dir,readLine)

100%|██████████| 2989/2989 [01:08<00:00, 43.72it/s]


In [ ]:
#preprocess zeroth
textList=glob(download_dir+zeroth_dir+"**/*.txt", recursive=True)
wavList=glob(download_dir+zeroth_dir+"**/*.flac", recursive=True)
working_dir=zeroth_dir

In [ ]:
def readLine(line,textPath):
    lineSplit = line.split(' ', 1)
    file_name=os.path.join("wavs", Path(lineSplit[0]).stem+ ".wav")
    text=lineSplit[1].strip()
    speaker_name="zeroth_"+Path(textPath).parent.name
    return [file_name,text,speaker_name]


saveResampleWav(wavList,save_dir,working_dir,target_rate)
saveCsv(textList,save_dir,working_dir,readLine)

 34%|███▎      | 7635/22720 [37:20<1:13:46,  3.41it/s]


KeyboardInterrupt: 

In [ ]:
#preprocess kss

textList=glob("kss/**/*.txt", recursive=True)
wavList=glob("kss/**/*.wav", recursive=True)
working_dir=kss_dir

In [ ]:
def readLine(line,textPath):
    lineSplit = line.split('|')
    file_name=os.path.join("wavs", Path(lineSplit[0]).stem+ ".wav")
    text=lineSplit[3].strip()
    speaker_name="kss"
    return [file_name,text,speaker_name]



saveResampleWav(wavList,save_dir,working_dir,target_rate)
saveCsv(textList,save_dir,working_dir,readLine)

# 3. Training

## 3-1. GlowTTS Training

In [ ]:
!pip install korean # resolved할 수 없다길래 설치

In [ ]:
import korean

def koreanCleaner(text):
    return "".join(korean.tokenize(text))

def koreanFormatter(root_path, meta_file):
    txt_file = os.path.join(root_path, meta_file)
    items = []
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, cols[0])
            text = cols[2].strip()
            text=koreanCleaner(text)
            speaker_name = cols[3].strip()
            items.append([text, wav_file, speaker_name])
    return items

In [ ]:
#from https://tts.readthedocs.io/en/latest/tutorial_for_nervous_beginners.html

import os

# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from TTS.trainer import Trainer, TrainingArgs

# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig, CharactersConfig, BaseAudioConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.utils.audio import AudioProcessor

# we use the same path as this script as our training folder.
output_path="./model/tts/glowtts/"

# DEFINE DATASET CONFIG
# Set LJSpeech as our target dataset and define its path.
# You can also use a simple Dict to define the dataset and pass it to your custom formatter.
dataset_config = [BaseDatasetConfig(name="kss", meta_file_train="metadata.csv", path=os.path.join("resample1","kss"))]


characters_config=CharactersConfig(
    pad='_',
    eos='~',
    bos='^',
    characters="".join(list(korean._punctuation)+ korean._letters),
    punctuations=korean._punctuation
)


# INITIALIZE THE TRAINING CONFIGURATION
# Configure the model. Every config class inherits the BaseTTSConfig.
config = GlowTTSConfig(
    batch_size=64,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="basic_cleaners",
    use_phonemes=False,
    use_espeak_phonemes=False,
    phoneme_language=None,
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=dataset_config,
    characters=characters_config,
)


# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor(**config.audio.to_dict())

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True, formatter=koreanFormatter)
eval_samples[:10]

In [ ]:
# INITIALIZE THE MODEL
# Models take a config object and a speaker manager as input
# Config defines the details of the model like the number of layers, the size of the embedding, etc.
# Speaker manager is used by multi-speaker models.
model = GlowTTS(config, speaker_manager=None)

# INITIALIZE THE TRAINER
# Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training,
# distributed training, etc.
trainer = Trainer(
    TrainingArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    training_assets={"audio_processor": ap},  # assets are objetcs used by the models but not class members.
)

# AND... 3,2,1... 🚀
trainer.fit()

## 3-2. Multi-band MalGan Training

In [ ]:
#vocoder

import os

from TTS.trainer import Trainer, TrainingArgs
from TTS.utils.audio import AudioProcessor
from TTS.vocoder.configs import HifiganConfig, MultibandMelganConfig
from TTS.vocoder.datasets.preprocess import load_wav_data
from TTS.vocoder.models.gan import GAN



output_path =os.path.join("./model/vocoder/mbmelgan/")

config = MultibandMelganConfig(
    batch_size=256,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=5,
    epochs=1000,
    seq_len=8192,
    pad_short=2000,
    use_noise_augment=True,
    eval_split_size=10,
    print_step=25,
    print_eval=False,
    mixed_precision=False,
    lr_gen=1e-4,
    lr_disc=1e-4,
    data_path="",
    output_path=output_path,
    steps_to_start_discriminator=0
)


# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# load training samples
eval_samples, train_samples=[[],[]]
for path in ["./resample1/kss/wavs","./resample1/pansori_tedxkr/wavs","./resample1/zeroth_korean/wavs"]:
    eval_samples_temp, train_samples_temp = load_wav_data(path, config.eval_split_size)
    eval_samples+=eval_samples_temp
    train_samples+=train_samples_temp

eval_samples

In [ ]:
# init model
model = GAN(config)

# init the trainer and 🚀
trainer = Trainer(
    TrainingArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    training_assets={"audio_processor": ap},
)
trainer.fit()

# 4. model test

In [ ]:
import korean
import IPython
import librosa
import librosa.display
import matplotlib.pyplot as plt

def koreanCleaner(text):
    return "".join(korean.tokenize(text))

def displayWaveImage(path):
    x, sr = librosa.load(path)
    plt.figure(figsize=(14, 5))
    librosa.display.waveplot(x, sr=sr)

ttsPath="model/tts/glowtts/coqui_tts-December-08-2021_03+15PM-0000000"
vocoderPath="model/vocoder/mbmelgan/coqui_tts-December-14-2021_05+54PM-0000000"

In [ ]:
#inference Using Griffin-Lim Vocoder
input_text = "신은 우리의 수학 문제에는 관심이 없다. 신은 다만 경험적으로 통합할 뿐이다."
inputText=koreanCleaner(input_text)

!tts \
    --text "$inputText" \
    --model_path "$ttsPath"/checkpoint_190000.pth.tar \
    --config_path "$ttsPath"/config.json \
    --out_path output.wav


displayWaveImage("output.wav")
IPython.display.Audio("output.wav")

In [ ]:
##inference Using trained Vocoder
input_text = "신은 우리의 수학 문제에는 관심이 없다. 신은 다만 경험적으로 통합할 뿐이다."
inputText=koreanCleaner(input_text)

!tts \
    --text "$inputText" \
    --model_path "$ttsPath"/checkpoint_190000.pth.tar \
    --config_path "$ttsPath"/config.json \
    --out_path output.wav \
    --vocoder_path "$vocoderPath"/checkpoint_150000.pth.tar \
    --vocoder_config_path "$vocoderPath"/config.json

displayWaveImage("output.wav")
IPython.display.Audio("output.wav")

In [ ]:
#tensorboard
!tensorboard --bind_all --logdir model/vocoder/mbmelgan/coqui_tts-December-14-2021_05+54PM-0000000